In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import seaborn as sns

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/fahmimnalfrzki/Additional-Materials/main/HousingPrices-Amsterdam-August-2021.csv').drop(columns='Unnamed: 0')
data

,Address,Zip,Price,Area,Room,Lon,Lat
0,"Blasiusstraat 8 2, Amsterdam",1091 CR,685000.0,64,3,4.907736,52.356157
1,"Kromme Leimuidenstraat 13 H, Amsterdam",1059 EL,475000.0,60,3,4.850476,52.348586
2,"Zaaiersweg 11 A, Amsterdam",1097 SM,850000.0,109,4,4.944774,52.343782
3,"Tenerifestraat 40, Amsterdam",1060 TH,580000.0,128,6,4.789928,52.343712
4,"Winterjanpad 21, Amsterdam",1036 KN,720000.0,138,5,4.902503,52.410538
...,...,...,...,...,...,...,...
919,"Ringdijk, Amsterdam",1097 AE,750000.0,117,1,4.927757,52.354173
920,"Kleine Beerstraat 31, Amsterdam",1033 CP,350000.0,72,3,4.890612,52.414587
921,"Stuyvesantstraat 33 II, Amsterdam",1058 AK,350000.0,51,3,4.856935,52.363256
922,"John Blankensteinstraat 51, Amsterdam",1095 MB,599000.0,113,4,4.965731,52.375268


In [ ]:
data_new = data.drop(columns=['Address','Zip'])
data_new

,Price,Area,Room,Lon,Lat
0,685000.0,64,3,4.907736,52.356157
1,475000.0,60,3,4.850476,52.348586
2,850000.0,109,4,4.944774,52.343782
3,580000.0,128,6,4.789928,52.343712
4,720000.0,138,5,4.902503,52.410538
...,...,...,...,...,...
919,750000.0,117,1,4.927757,52.354173
920,350000.0,72,3,4.890612,52.414587
921,350000.0,51,3,4.856935,52.363256
922,599000.0,113,4,4.965731,52.375268


In [ ]:
data_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 924 entries, 0 to 923
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Price   920 non-null    float64
 1   Area    924 non-null    int64  
 2   Room    924 non-null    int64  
 3   Lon     924 non-null    float64
 4   Lat     924 non-null    float64
dtypes: float64(3), int64(2)
memory usage: 36.2 KB


In [ ]:
data_new.dropna(inplace=True)

In [ ]:
data_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 920 entries, 0 to 923
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Price   920 non-null    float64
 1   Area    920 non-null    int64  
 2   Room    920 non-null    int64  
 3   Lon     920 non-null    float64
 4   Lat     920 non-null    float64
dtypes: float64(3), int64(2)
memory usage: 43.1 KB


In [ ]:
data_new.corr()

,Price,Area,Room,Lon,Lat
Price,1.000000,0.835090,0.623448,-0.013561,0.062196
Area,0.835090,1.000000,0.812421,0.012196,0.017993
Room,0.623448,0.812421,1.000000,-0.036585,-0.016381
Lon,-0.013561,0.012196,-0.036585,1.000000,-0.184436
Lat,0.062196,0.017993,-0.016381,-0.184436,1.000000


In [ ]:
sns.heatmap(data_new.corr())

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_new[['Area','Room','Lon','Lat']], data_new['Price'], train_size=0.8)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model = LinearRegression(fit_intercept=True)
model.fit(X_train_scaled,y_train)

print(model.intercept_)
print(model.coef_)

623048.8111413092
[466385.24007387 -72558.05735481 -11920.45565078  25144.6630959 ]


In [ ]:
import pandas as pd

metric = pd.DataFrame()
metric['models'] = ['OLR','Poly3','OLR_reg','Poly3_reg']
r2_train = []
r2_test = []

## Ordinary Linear Regression

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print('-------- TRAIN ---------')
y_pred = model.predict(X_train_scaled)
print('MAE:',mean_absolute_error(y_train,y_pred))
print('MSE:',mean_squared_error(y_train,y_pred))
print('RMSE:',mean_squared_error(y_train,y_pred,squared=False))
print('R^2 score:',r2_score(y_train,y_pred))
r2_train.append(r2_score(y_train,y_pred))

print('-------- TEST ---------')
y_pred = model.predict(X_test_scaled)
print('MAE:',mean_absolute_error(y_test,y_pred))
print('MSE:',mean_squared_error(y_test,y_pred))
print('RMSE:',mean_squared_error(y_test,y_pred,squared=False))
print('R^2 score:',r2_score(y_test,y_pred))
r2_test.append(r2_score(y_test,y_pred))

-------- TRAIN ---------
MAE: 170891.55974385038
MSE: 73651043442.76848
RMSE: 271387.25733307464
R^2 score: 0.696774722354337
-------- TEST ---------
MAE: 185325.3425003142
MSE: 134800764858.7571
RMSE: 367152.2366250233
R^2 score: 0.718817537219296


## Polynomial Features 3-degree Linear Regression

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(3)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

scaler = StandardScaler()
scaler.fit(X_train_poly)

X_train_scaled = scaler.transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

model = LinearRegression(fit_intercept=True)
model.fit(X_train_scaled,y_train)

print('-------- TRAIN ---------')
y_pred = model.predict(X_train_scaled)
print('MAE:',mean_absolute_error(y_train,y_pred))
print('MSE:',mean_squared_error(y_train,y_pred))
print('RMSE:',mean_squared_error(y_train,y_pred,squared=False))
print('R^2 score:',r2_score(y_train,y_pred))
r2_train.append(r2_score(y_train,y_pred))

print('-------- TEST ---------')
y_pred = model.predict(X_test_scaled)
print('MAE:',mean_absolute_error(y_test,y_pred))
print('MSE:',mean_squared_error(y_test,y_pred))
print('RMSE:',mean_squared_error(y_test,y_pred,squared=False))
print('R^2 score:',r2_score(y_test,y_pred))
r2_test.append(r2_score(y_test,y_pred))

-------- TRAIN ---------
MAE: 116344.87373187288
MSE: 43306361166.16411
RMSE: 208101.80481236608
R^2 score: 0.8217053992094825
-------- TEST ---------
MAE: 150009.780380252
MSE: 115111527126.82855
RMSE: 339280.8970850386
R^2 score: 0.7598875442147257


## Ordinary Linear Regression + Regularization

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = Ridge(alpha=10, fit_intercept=True)
model.fit(X_train_scaled,y_train)

print('-------- TRAIN ---------')
y_pred = model.predict(X_train_scaled)
print('MAE:',mean_absolute_error(y_train,y_pred))
print('MSE:',mean_squared_error(y_train,y_pred))
print('RMSE:',mean_squared_error(y_train,y_pred,squared=False))
print('R^2 score:',r2_score(y_train,y_pred))
r2_train.append(r2_score(y_train,y_pred))

print('-------- TEST ---------')
y_pred = model.predict(X_test_scaled)
print('MAE:',mean_absolute_error(y_test,y_pred))
print('MSE:',mean_squared_error(y_test,y_pred))
print('RMSE:',mean_squared_error(y_test,y_pred,squared=False))
print('R^2 score:',r2_score(y_test,y_pred))
r2_test.append(r2_score(y_test,y_pred))

-------- TRAIN ---------
MAE: 170070.49838455132
MSE: 73779291800.68892
RMSE: 271623.4375025265
R^2 score: 0.6962467170183604
-------- TEST ---------
MAE: 186146.0759103599
MSE: 137343013986.0854
RMSE: 370598.1840026815
R^2 score: 0.7135146305824288


## Polynomial Features 3-degree Linear Regression + Regularization

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(3)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

scaler = StandardScaler()
scaler.fit(X_train_poly)

X_train_scaled = scaler.transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

model = Ridge(alpha=10,fit_intercept=True)
model.fit(X_train_scaled,y_train)

print('-------- TRAIN ---------')
y_pred = model.predict(X_train_scaled)
print('MAE:',mean_absolute_error(y_train,y_pred))
print('MSE:',mean_squared_error(y_train,y_pred))
print('RMSE:',mean_squared_error(y_train,y_pred,squared=False))
print('R^2 score:',r2_score(y_train,y_pred))
r2_train.append(r2_score(y_train,y_pred))

print('-------- TEST ---------')
y_pred = model.predict(X_test_scaled)
print('MAE:',mean_absolute_error(y_test,y_pred))
print('MSE:',mean_squared_error(y_test,y_pred))
print('RMSE:',mean_squared_error(y_test,y_pred,squared=False))
print('R^2 score:',r2_score(y_test,y_pred))
r2_test.append(r2_score(y_test,y_pred))

-------- TRAIN ---------
MAE: 161506.04454112923
MSE: 68072866381.39797
RMSE: 260907.77370825494
R^2 score: 0.719740375101743
-------- TEST ---------
MAE: 178321.72268582723
MSE: 137221384367.83154
RMSE: 370434.0486076186
R^2 score: 0.7137683391993164


In [ ]:
metric['R2_Train'] = r2_train
metric['R2_Test'] = r2_test

metric

,models,R2_Train,R2_Test
0,OLR,0.696775,0.718818
1,Poly3,0.821705,0.759888
2,OLR_reg,0.696247,0.713515
3,Poly3_reg,0.734148,0.709883
